# 🔬 4D Tesseract Visualizer
Visualize a 4D hypercube projected into 3D and 2D, with interactive sliders for rotating in all six 4D planes. Because cubes are boring.

In [ ]:
%pip install numpy matplotlib ipywidgets

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Line3DCollection
from matplotlib.lines import Line2D
from itertools import product
from ipywidgets import interact, FloatSlider, fixed

In [ ]:
EPSILON = 1e-6
AXIS_COLORS = ['orange', 'green', 'blue', 'red']
AXIS_LABELS = ['X', 'Y', 'Z', 'W']

In [ ]:
def generate_hypercube_vertices():
    return np.array(list(product([-0.5, 0.5], repeat=4)))

def generate_hypercube_edges(vertices):
    edges = []
    for i in range(len(vertices)):
        for j in range(i + 1, len(vertices)):
            if np.isclose(np.sum(np.abs(vertices[i] - vertices[j])), 1.0):
                edges.append((i, j))
    return edges

In [ ]:
def rotation_matrix_4d(axis1, axis2, angle_rad):
    M = np.eye(4)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    M[axis1, axis1], M[axis2, axis2] = c, c
    M[axis1, axis2], M[axis2, axis1] = -s, s
    return M

In [ ]:
def project_4d_to_3d(points4d, viewer_distance=3.0):
    w = points4d[:, 3]
    denom = np.where(np.abs(viewer_distance - w) < EPSILON, EPSILON, viewer_distance - w)
    factor = viewer_distance / denom
    return points4d[:, :3] * factor[:, np.newaxis]

In [ ]:
def get_edge_color(vertices, edge):
    diff = vertices[edge[0]] - vertices[edge[1]]
    axis = np.flatnonzero(diff)[0]
    return AXIS_COLORS[axis]

In [ ]:
def plot_tesseract(rot_xy=0, rot_xz=0, rot_xw=0, rot_yz=0, rot_yw=0, rot_zw=0, viewer_distance=3.0, show_plot=True):
    verts = generate_hypercube_vertices()
    edges = generate_hypercube_edges(verts)

    R = (
        rotation_matrix_4d(2, 3, rot_zw) @
        rotation_matrix_4d(1, 3, rot_yw) @
        rotation_matrix_4d(0, 3, rot_xw) @
        rotation_matrix_4d(1, 2, rot_yz) @
        rotation_matrix_4d(0, 2, rot_xz) @
        rotation_matrix_4d(0, 1, rot_xy)
    )

    rv = verts @ R.T
    p3 = project_4d_to_3d(rv, viewer_distance)

    ecols = [get_edge_color(verts, e) for e in edges]
    wvals = rv[:, 3]
    wmin, wmax = wvals.min(), wvals.max()
    wnorm = (wvals - wmin) / (wmax - wmin) if not np.isclose(wmax, wmin) else np.full_like(wvals, 0.5)
    sizes = 20 + 40 * wnorm

    fig = plt.figure(figsize=(15, 8))
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.set_box_aspect((1, 1, 1))
    M = np.max(np.abs(p3)) * 1.15
    ax1.set(xlim=(-M, M), ylim=(-M, M), zlim=(-M, M),
           xlabel='X', ylabel='Y', zlabel='Z', title='3D Projection')
    lines = [p3[list(e)] for e in edges]
    ax1.add_collection3d(Line3DCollection(lines, colors=ecols, linewidths=1.5, alpha=0.9))
    ax1.scatter(p3[:, 0], p3[:, 1], p3[:, 2], c=wvals, s=sizes, cmap='viridis', edgecolors='black')

    ax2 = fig.add_subplot(122)
    ax2.set_aspect('equal')
    ax2.set(xlabel='X', ylabel='Y', title='2D Projection (X vs Y)')
    for idx, (i, j) in enumerate(edges):
        p1, p2 = p3[i], p3[j]
        ax2.plot([p1[0], p2[0]], [p1[1], p2[1]], color=ecols[idx], alpha=0.8, linewidth=1.2)
    ax2.scatter(p3[:, 0], p3[:, 1], c=wvals, s=sizes, cmap='viridis', edgecolors='black')

    plt.tight_layout()
    if show_plot:
        plt.show()

In [ ]:
interact(
    plot_tesseract,
    rot_xy=FloatSlider(min=-np.pi, max=np.pi, step=0.05, value=0, description='XY (3D)'),
    rot_xz=FloatSlider(min=-np.pi, max=np.pi, step=0.05, value=0, description='XZ (3D)'),
    rot_yz=FloatSlider(min=-np.pi, max=np.pi, step=0.05, value=0, description='YZ (3D)'),
    rot_xw=FloatSlider(min=-np.pi, max=np.pi, step=0.05, value=0, description='XW (4D)'),
    rot_yw=FloatSlider(min=-np.pi, max=np.pi, step=0.05, value=0, description='YW (4D)'),
    rot_zw=FloatSlider(min=-np.pi, max=np.pi, step=0.05, value=0, description='ZW (4D)'),
    viewer_distance=FloatSlider(min=2.0, max=10.0, step=0.1, value=3.0, description='Viewer Dist'),
    show_plot=fixed(True))